In [1]:
import pandas as pd

fruits_data = {
    '날짜': ['2023-01-01', '2023-01-01', '2023-01-01', '2023-01-01', '2023-01-02', '2023-01-02', '2023-01-02', '2023-01-02'],
    '지역': ['서울', '서울', '부산', '부산', '서울', '서울', '대구', '대구'],
    '과일': ['사과', '사과', '딸기', '딸기', '사과', '사과', '참외', '참외'],
    '판매량': [100, 80, 50, 30, 120, 100, 40, 30]
}

fruits_df = pd.DataFrame(fruits_data)

df = fruits_df.set_index(['날짜', '지역'])
df

과일  판매량
날짜         지역         
2023-01-01 서울  사과  100
           서울  사과   80
           부산  딸기   50
           부산  딸기   30
2023-01-02 서울  사과  120
           서울  사과  100
           대구  참외   40
           대구  참외   30

In [2]:
# 지역 별 판매량 합계를 날짜 별로 보기
grouped_s = df.groupby(['날짜', '지역'])['판매량'].sum()
grouped_s

날짜          지역
2023-01-01  부산     80
            서울    180
2023-01-02  대구     70
            서울    220
Name: 판매량, dtype: int64

In [3]:
type(grouped_s)

pandas.core.series.Series

In [4]:
# grouped_df = df.groupby(['날짜', '지역'])[['판매량']].sum()
# type(grouped_df)

In [5]:
# 0: 날짜, 1: 지역
grouped_s.index

MultiIndex([('2023-01-01', '부산'),
            ('2023-01-01', '서울'),
            ('2023-01-02', '대구'),
            ('2023-01-02', '서울')],
           names=['날짜', '지역'])

In [6]:
grouped_s

날짜          지역
2023-01-01  부산     80
            서울    180
2023-01-02  대구     70
            서울    220
Name: 판매량, dtype: int64

In [7]:
# '지역'을 인덱스에서 컬럼으로 이동
# 동일 표현: unstack(), unstack(1), unstack('지역')
unstacked_df = grouped_s.unstack(level=1)
unstacked_df

지역,대구,부산,서울
날짜,,,
2023-01-01,NaN,80.0,180.0
2023-01-02,70.0,NaN,220.0


In [8]:
unstacked_df.columns

Index(['대구', '부산', '서울'], dtype='object', name='지역')

In [9]:
# '지역'을 컬럼에서 인덱스로 이동
# 동일 표현: stack(), stack(0), stack('지역')
stacked_s = unstacked_df.stack(level=0)
stacked_s

날짜          지역
2023-01-01  부산     80.0
            서울    180.0
2023-01-02  대구     70.0
            서울    220.0
dtype: float64

In [10]:
stacked_s

날짜          지역
2023-01-01  부산     80.0
            서울    180.0
2023-01-02  대구     70.0
            서울    220.0
dtype: float64

In [11]:
stacked_s.index

MultiIndex([('2023-01-01', '부산'),
            ('2023-01-01', '서울'),
            ('2023-01-02', '대구'),
            ('2023-01-02', '서울')],
           names=['날짜', '지역'])

In [12]:
# 인덱스 리셋
# reset_df = stacked_s.reset_index(0) # '지역' 인덱스만 리셋
# reset_df = stacked_s.reset_index(level=1) # '날짜' 인덱스만 리셋
reset_df = stacked_s.reset_index() # 모든 인덱스 리셋
reset_df

,날짜,지역,0
0,2023-01-01,부산,80.0
1,2023-01-01,서울,180.0
2,2023-01-02,대구,70.0
3,2023-01-02,서울,220.0


In [13]:
reset_df = reset_df.rename(columns = { 0 : '판매량' })
reset_df

,날짜,지역,판매량
0,2023-01-01,부산,80.0
1,2023-01-01,서울,180.0
2,2023-01-02,대구,70.0
3,2023-01-02,서울,220.0


In [14]:
# '지역' 인덱스만 리셋
# reset_df = stacked_s.reset_index(0)
# reset_df = stacked_s.reset_index(level=0)
# reset_df = stacked_s.reset_index('지역')
# reset_df

In [15]:
fruits_df

,날짜,지역,과일,판매량
0,2023-01-01,서울,사과,100
1,2023-01-01,서울,사과,80
2,2023-01-01,부산,딸기,50
3,2023-01-01,부산,딸기,30
4,2023-01-02,서울,사과,120
5,2023-01-02,서울,사과,100
6,2023-01-02,대구,참외,40
7,2023-01-02,대구,참외,30


In [16]:
# (날짜, 과일) 인덱스에 중복된 값이 있을 때 pivot 사용불가
# pivot_df = fruits_df.pivot(index='날짜', columns='과일', values='판매량')

In [17]:
# (날짜, 과일) 인덱스에 중복된 값이 있을 때 pivot_table 사용가능
pivot_table_df = fruits_df.pivot_table(index='날짜', columns='과일', values='판매량', aggfunc='sum')
pivot_table_df

과일,딸기,사과,참외
날짜,,,
2023-01-01,80.0,180.0,NaN
2023-01-02,NaN,220.0,70.0


In [18]:
pivot_table_df

과일,딸기,사과,참외
날짜,,,
2023-01-01,80.0,180.0,NaN
2023-01-02,NaN,220.0,70.0


In [19]:
pivot_table_df.index

Index(['2023-01-01', '2023-01-02'], dtype='object', name='날짜')

In [20]:
melt_df = pivot_table_df.reset_index().melt(id_vars='날짜', var_name='과일', value_name='판매량')
melt_df

,날짜,과일,판매량
0,2023-01-01,딸기,80.0
1,2023-01-02,딸기,NaN
2,2023-01-01,사과,180.0
3,2023-01-02,사과,220.0
4,2023-01-01,참외,NaN
5,2023-01-02,참외,70.0


In [21]:
melt_df.index

RangeIndex(start=0, stop=6, step=1)